### Implementation of the linear regression model:
$ y=b_{0}+b_{1}x $ .

The optimalization algorithm relies on traditional gradient-finding.
I have implemend two method of fitting the linear regression. The first is described above, the second is Adaptive Moment (Adam) optimalization algorithm.

In [119]:
import numpy as np
import polars as pl
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler


def MSE(X: np.ndarray, y: np.ndarray, b0: float, b1:float) -> float:
    """The function computes the MeanSquareError for the linear regression model"""

    return np.sum((b1*X + b0 - y)**2)/X.shape[0]

     

def FitTheLineV1(dataset:pl.DataFrame, predictor:str, target_var:str, eps = 0.001) -> tuple[np.ndarray, float]:
    """The function finds the optimal parameters, b0, b1, so that 
    the loss function of linear regressionmodel dataset[target_var] = b0 + b1*dataset[predictor] is minimized"""

    np.random.seed(1)
    B = np.random.uniform(low = -3, high = 3, size = 2)

    #Take the X feature and y feature from the dataset. Then, numpyize it!. Afterward, normalize it!
    MinMaxScaler_X = MinMaxScaler()
    MinMaxScaler_y = MinMaxScaler()

    X = MinMaxScaler_X.fit_transform(dataset.select(predictor).to_numpy())
    y = MinMaxScaler_y.fit_transform(dataset.select(target_var).to_numpy())


    #Define the h value for computing derivatives and learning-rate (lr) for updating b0, b1 parameters.
    h = 0.001
    lr = 0.01
    #The following variable indicates the difference between the new value of the function and the old one.
    #If it's  absolute is less than eps, break the loop.
    val_change= float('inf')

    while val_change  >= eps:
        #Compute the gradients for b0 and b1 variables respectively.
        prev_MSE_val= MSE(X, y, B[0], B[1])

        grad_b0 = (MSE(X, y,  B[0]+h, B[1]) - prev_MSE_val)/h
        grad_b1 = (MSE(X, y,  B[0], B[1]+h) - prev_MSE_val)/h

        gradient = np.array([grad_b0, grad_b1])

        B = B - lr * gradient

        curr_MSE_val = MSE(X, y, B[0], B[1])
        val_change = abs(curr_MSE_val - prev_MSE_val)


    return B, MSE(X, y, B[0],B[1])

        

#Read the dataset, from which we take the features.
dataset = pl.read_csv("WineQuality.csv", separator=';', ignore_errors = True)


#Select the features for the model.
target_var = "quality"
predictor = "pH"


print(FitTheLineV1(dataset, predictor, target_var))


(array([-0.25404805,  1.41196348]), 0.08011607540401058)


### Adam optimalization algorithm.

In [120]:
import numpy as np
import polars as pl
from sklearn.preprocessing import MinMaxScaler

class Adam():
        
    #Note, we assume obj_func takes two, equally-shaped, one-dimensional arrays and two parameters: b0 and b1 (thus n_params = 2)
    def __init__(self,  params:np.ndarray,  X: np.ndarray, y: np.ndarray, alpha:float = 0.001, eps: float = 10**(-8)):
        assert X.shape[0] == y.shape[0], "The first dimensions of the arrays should match!"

        assert type(alpha) is float and alpha > 0, "The alpha parameter needs to be a positive, numerical value."

    
        self.params = params
        self.n_params = params.shape[0]
        
        #Declare the X variable and y variable.
        self.X = X
        self.y = y

        #Define the alpha and eps which stands for: learning-rate and zero-division-preventer respectively.
        self.alpha = alpha
        self.eps = eps

        #Declare the initial state of m and v (first-moment gradient, second-moment gradient).
        self.m = np.zeros(self.n_params)
        self.v = np.zeros(self.n_params)

    def MSE(self, b0: float, b1:float) -> float:
        return np.sum((b1*self.X + b0 - self.y)**2)/self.X.shape[0]


    def FindGradients(self,h:float) -> np.ndarray:
        partial_derivs = np.zeros(self.n_params)
        substractor = self.MSE(self.params[0], self.params[1])

        partial_derivs[0] = (self.MSE(self.params[0]+h, self.params[1]) - substractor)/h
        partial_derivs[1] = (self.MSE(self.params[0], self.params[1]+h) - substractor)/h

        return partial_derivs


    
    def UpdateMoments(self, bet1:float, bet2:float, h:float):
        Gradients = self.FindGradients(h)

        new_m = (bet1*self.m + (1-bet1)*Gradients)/(1-bet1)
        new_v = (bet2*self.v + (1-bet2)*Gradients**2)/(1-bet2)


        return new_m, new_v
    
    def UpdateParameters(self, bet1:float, bet2:float, h:float):
        m_corr, v_corr = self.UpdateMoments(bet1, bet2, h)

        self.params = self.params - self.alpha * m_corr / ( np.sqrt(v_corr) +self.eps)

        return self.params
    
    



def FitTheLineV2(dataset:pl.DataFrame, predictor:str, target_var:str, eps = 0.001) -> tuple[np.ndarray, float]:
    """The function finds the optimal parameters, b0, b1, so that 
    the loss function of linear regressionmodel dataset[target_var] = b0 + b1*dataset[predictor] is minimized"""

    np.random.seed(1)
    B = np.random.uniform(low = -3, high = 3, size = 2)

    #Take the X feature and y feature from the dataset. Then, numpyize it!. Afterward, normalize it!
    MinMaxScaler_X = MinMaxScaler()
    MinMaxScaler_y = MinMaxScaler()

    X = MinMaxScaler_X.fit_transform(dataset.select(predictor).to_numpy())
    y = MinMaxScaler_y.fit_transform(dataset.select(target_var).to_numpy())


    #Declare an instance of the Adam optimizer.
    Adamek = Adam(B, X, y)

    #The following variable indicates the difference between the new value of the function and the old one.
    #If it's  absolute is less than eps, break the loop.
    val_change= float('inf')

    while val_change  >= eps:
        B_new = Adamek.UpdateParameters(0.9, 0.9, 0.001)

        val_change = abs( Adamek.MSE(B_new[0], B_new[1]) - Adamek.MSE(B[0], B[1]))
        B = B_new



    return B, Adamek.MSE(B[0], B[1])

        


FitTheLineV2(dataset, predictor, target_var, eps = 0.0005)



(array([-0.32986797,  1.48994695]), 0.09721925361635286)